# Data Matching Functions

This notebook contains reusable functions that match max value disease pairs with data from matrices. Max value disease pair files include:

    Max Gene Uniqueness.xlsx
    Max BP-Weight.xlsx
    Max BP Jaccard Similarity.xlsx

The functions can retrieve data stored in half-full matrices. These matrices include:

    BP-Weight.csv
    Gene Uniqueness.csv
    BP Jaccard Similarity.csv

## Define a function that opens max measurement files stored as Excel files

In [ ]:
def open_max_excel_file(filename, sheet = 'No Duplicates'):
    '''Open the file with the max disease pairs. By default,
    the sheet with no duplicates will be used. 
    
    Parameters:
    filename (str): The filename to work with.
    sheet (str): The sheet name.
    '''
    print("Opening Excel file.")
    file = pandas.read_excel(filename,      # Specify filename.
                             dtype = str,   # Treat cells as strings.
                             sheet_name = sheet, # The sheet to open.
                             index_col = 0) # Use column 0 as index.
    
    # Return the opened file object.
    return file

## Define a function that opens .csv matrix files and used DB IDs to locate values

The matrices should have the following format:

|            | DB ID  | Disease| 0   | 1   | 2   | ... |
|   --       | --     |  --    |  -- | --  |  -- |  -- |   
| **0**      | 000001 | Name 1 | 2.5 | 0.8 | 0.3 | ... |
| **1**      | 000002 | Name 2 | NaN | 1.2 | 0.2 | ... |
| **2**      | 000003 | Name 3 | NaN | NaN | 0.5 | ... |
| **..**     | ...    | ...    | ... | ... | ... | ... |
| **DB ID**  | NaN | NaN | 000001 | 000002 | 000003 | ... |
| **Disease**| NaN | NaN | Name 1 | Name 2 | Name 3 | ... |

The example matrix includes the columns 'DB ID' and 'Disease'. The program should not fail if matrices have additional columns, or if matrices do not have the 'Disease' column. However, the program will fail if the 'DB ID' column does not exist.

The function output looks like this:

| **DB ID**  | 000001   | 000002   | 000003   | ... |
|   --       |  -- | --  |  -- |  -- |   
| **000001** | NaN | 0.8 | 0.3 | ... |
| **000002** | NaN | NaN | 0.2 | ... |
| **000003** | NaN | NaN | NaN | ... |
| **...**     | ... | ... | ... | ... |

In [ ]:
def open_file(filename):
    '''Open .csv matrix file. The matrix must have a 'DB ID' column 
    and a '0' column. 
    
    Parameters:
    filename (str): The filename to work with.
    '''
    
    print("Opening .csv file.")
    file = pandas.read_csv(filename,      # Specify filename.
                           dtype = str,   # Treat cells as strings.
                           index_col = 0) # Use column 0 as index.
    
    # Everything that comes before column '0' is unused, for example:
    # If location for '0' is 2, the labels at 0 and 1 must be removed.
    unused_columns = file.columns.get_loc('0')

    # Convert database IDs to floats. Example: '100.0' becomes 100.0.
    db_ids = file['DB ID'].astype(float)
    
    # Drop NaN values and convert to int. Example: 100.0 becomes 100.
    db_ids = db_ids.dropna().astype(int)
    
    # Convert integers to strings, for example, 100 becomes '100'.
    # This removes decimals from strings: '100.0' to '100'.
    index_to_db_id = db_ids.astype(str).to_dict()

    # Get labels that come before column '0' for removal. Example:
    # the labels 'DB ID' and 'Disease' may come before '0'.
    labels_to_drop = file.columns[range(unused_columns)]

    print('Dropping unused rows and columns.')
    # Remove the first unused columns.
    file = file.drop(columns = labels_to_drop)
    # Remove the last unused rows.
    file = file.drop(index = labels_to_drop)

    print('Replacing labels with corresponding DB IDs.')
    # Rename the columns using corresponding database IDs.
    file = file.rename(columns = index_to_db_id)
    # Rename the rows using corresponding database IDs.
    file = file.rename(index = index_to_db_id)
    
    # Label index column using the name 'DB ID'.
    file.index.name = 'DB ID'
    
    print('Converting values to type float.')
    # Convert values to floats; it's difficult to do math on strings.
    file = file.astype(float)
    
    # Return the opened file object.
    return file

## Define a function that uses DB IDs to retrieve values from a matrix

In [3]:
def find_values_using_database_ids(db_ids1, db_ids2, matrix):
    '''Return table containing values corresponding to specified 
    DB ID pairs. 
    
    Parameters:
    db_ids1 (list): List of DB IDs pointing at matrix rows/columns.
    db_ids2 (list): List of DB IDs pointing at matrix rows/columns.
    matrix: Pandas dataframe using DB IDs for the index and the column
    names.
    '''
    # Define an empty dictionary to store values specified by DB IDs.
    values = {}
    
    # Iterate thru every DB ID pair formed by using both lists.
    # Each DB ID pair is a coordinate that points at a matrix cell.
    for id1, id2 in zip(db_ids1, db_ids2):
        
        try:
            
            # Try to use DB ID to find the value in the matrix file.
            # Matrix is half full; finding the correct value depends
            # on pointing to the correct half; one half is empty.
            
            # Get the first value from one half of the matrix.
            similarity1 = matrix.at[id1, id2]
            # Get the second value from the other half of the matrix.
            similarity2 = matrix.at[id2, id1]
            
            # Convert NaN values to 0 to perform math on variables.
            similarity1 = 0 if numpy.isnan(similarity1) else similarity1
            similarity2 = 0 if numpy.isnan(similarity2) else similarity2
            
        except ValueError:
            
            # Cannot label index with a null key; one of the DB IDs is
            # null, which means other DB ID corresponds to a disease 
            # without similarity to other disease; similarity is NaN.
            values[id1] = numpy.nan
            
            # Process the next set of database IDs.
            continue

        # Store the similarity value that is greater than 0 (not NaN).
        values[id1] = max(similarity1, similarity2)
    
    # Convert the dictionary to a pandas dataframe.
    values = pandas.DataFrame.from_dict(values, orient = 'index')
    
    # Return table containing values corresponding to given DB IDs.
    return values

## Define a function that takes a table and stores values obtained from matrices

In [4]:
def get_values_from_matrices(target, paths, names):
    '''Return table after adding values extracted from matrices. 
    
    Parameters:
    target: Pandas dataframe containing two columns named 'DB ID 1' 
    and 'DB ID 2'.
    paths (list): List of file paths that point to matrices containing
    data about disease pairs.
    names (list): List of names that will be used as column names once
    the data obtained from the matrices is placed in the target file.
    '''
    # Create a copy of the pandas dataframe.
    file = target.copy()
    
    # Get the first list of DB IDs.
    db_id1 = file['DB ID 1']
    # Get the second list of DB IDs.
    db_id2 = file['DB ID 2']
    
    # Convert database IDs to floats. Example: '100.0' becomes 100.0.
    # Convert IDs to int. Example: 100.0 becomes 100.
    # Convert integers to strings, for example, 100 becomes '100'.
    # This removes decimals from strings: '100.0' to '100'.
    db_id1 = db_id1.astype(float).astype(int).astype(str).to_list()
    db_id2 = db_id2.astype(float).astype(int).astype(str).to_list()
    
    # Iterate thru every file path and its corresponding column name.
    for path, name in zip(paths, names):
        
        # Print name of the column and the file path for debugging.
        print(name, '\t', path)
        
        # Get a matrix using the specified file path; the matrix will
        # use DB IDs for the index and for columns, so values can be 
        # found using DB IDs.
        matrix = open_file(path)
        
        # Use the lists of DB IDs obtained from the target file to
        # locate the values stored in the matrix file.
        values = find_values_using_database_ids(
            db_id1, db_id2, matrix)
        
        # Store the values corresponding to the DB IDs in the copy
        # of the target file.
        file[name] = values.values
    
    # Return table after adding values extracted from the matrices.
    return file

## Print the functions included in the notebook

In [5]:
print('''Define open_max_excel_file: Open the file with the max disease pairs. 
Define open_file: Open .csv matrix file.
Define find_values_using_database_ids: Return table corresponding to specified DB ID pairs. 
Define get_values_from_matrices: Return values extracted from matrices.''')

Define open_max_excel_file: Open the file with the max disease pairs. 
Define open_file: Open .csv matrix file.
Define find_values_using_database_ids: Return table corresponding to specified DB ID pairs. 
Define get_values_from_matrices: Return values extracted from matrices.
